In [79]:
import numpy as np
import xgboost as xgb
import pandas as pd
import category_encoders as ce
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels
import csv


In [72]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

id_column = train_data

train_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]

X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_train.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 1.0, inplace=True)
y_train_numeric.replace('Charged Off', 0.0, inplace=True)


In [92]:
xg_train, xg_test, xg_ytrain, xg_ytest = sklearn.model_selection.train_test_split(
    X_train_numeric, y_train_numeric, test_size=0.5, random_state=0)
print(xg_train.shape, xg_test.shape, xg_ytrain.shape, xg_ytest.shape)

(98625, 13) (98625, 13) (98625,) (98625,)


In [93]:
def auc(m, xtrain, xtest): 
    return (metrics.roc_auc_score(xg_ytrain,m.predict_proba(xtrain)[:,1], average='weighted'),
                            metrics.roc_auc_score(xg_ytest,m.predict_proba(xtest)[:,1], average='weighted'))

# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [5],
              "learning_rate": [0.05, 0.1,0.16],}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(xg_train, xg_ytrain)

grid_search.best_estimator_

model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=10,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(xg_train,xg_ytrain)
predictions = model.predict_proba(xg_test)[:,1]

auc(model, xg_train, xg_test)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:49:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:49:23] WARNING: ../src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:49:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.9904785601627684, 0.6380615901023039)

In [94]:

# def auc2(m, train, test): 
#     return (metrics.roc_auc_score(lgb_y_train,m.predict(lgb_X_train),multi_class='ovr'),
#                             metrics.roc_auc_score(y_train_numeric,m.predict(lgb_x_test), multi_class='ovr'))

# lg = lgb.LGBMClassifier(silent=False)
# param_dist = {"max_depth": [25,50, 75],
#               "learning_rate" : [0.01,0.05,0.1],
#               "num_leaves": [300,900,1200],
#               "n_estimators": [5]
#              }
# grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
# grid_search.fit(lgb_X_train,lgb_y_train)
# grid_search.best_estimator_

# d_train = lgb.Dataset(X_train_numeric, label=y_train_numeric)
# params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 5}

# # # Trying Categorical Features
# model2 = lgb.train(params, d_train)
# auc2(model2, lgb_x_test, lgb_y_test)
# # roc = {label: [] for label in multi_class_series.unique()}
# # for label in multi_class_series.unique():
# #     model2.fit(X_train_numeric, y_train_numeric == label)
# #     predictions_proba = selected_classifier.predict_proba(X_test_numeric)
# #     roc[label] += metrics.roc_auc_score(y_test, predictions_proba[:,1])

# #Trying without Catgeorical Features
# # cate_features_name = ["subgrade","emp_length","home_ownership","verification_status","issue_d",
# #                  "purpose","zipcode","addr_state", "earliest_cr_line", "initial_list_status",
# #                  "application_type", "loan_status"]
# # model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
# # auc2(model2, X_test_numeric, y_test)

<ipython-input-5-dc2f71e5faf5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col]=pd.Categorical(X[col],categories=train_X[col].dropna().unique())


TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:

# clf = xgb.XGBClassifier(
#     tree_method="gpu_hist", enable_categorical=True, use_label_encoder=False
# )
# # X is the dataframe we created in previous snippet
# clf.fit(X, y)

KeyError: 'cat_feature'